<a href="https://colab.research.google.com/github/Emmanuel1989/MNP-Final-Proyect/blob/main/ejercicio_6_del_examen_MNP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pingouin


In [ ]:
# La prueba no paramétrica basada en permutaciones se basa en el reordenamiento de los
# datos para determinar que tan inusual es el valor del estadístico de prueba obtenido. Es
# utilizado cuando no se tiene certeza de la normalidad en las observaciones. Como ejemplo
# considere el siguiente conjunto de datos:
# | Treatment 1   |  Treatment 2 |
# |       y1j     |      y2j     |
# |        3      |       8      |
# |        7      |       10     |
# Aquí, el valor del estadístico F para el ANOVA de un factor es F=3.20
# Tomando las posibles permutaciones, revisamos los valores del estadístico:
# | Randomization |  Treatment 1 |  Treatment 2 |      F*     |  Probability |
# |        1      |      3.7     |      8.10    |    3.20     |      1/6     |
# |        2      |       3.8    |       7.10   |    1.06     |      1/6     |
# |        3      |       3.10   |      8.7     |    0.08     |      1/6     |
# |        4      |      8.7     |       3.10   |    0.08     |      1/6     |
# |        5      |      7.10    |       3.8    |    1.06     |      1/6     |
# |        6      |       8.10   |       3.7    |    3.20     |      1/6     |
# Y con estos valores, observamos la probabilidad de rechazar con esa F,
# P(F* >= 3.20) = 2/6 = 0.33
# No rechazando la hipótesis nula.
# Escriba un código para obtener la prueba de permutación para un ANOVA con 3 factores y
# 10 réplicas. Haga un ejemplo con los siguientes escenarios
# a) x1 = rt(10, 3)+10; x2 = rt(10, 3)+10; x3 = rt(10,3)+10
# b) x1 = rt(10, 3)+10; x2 = rt(10, 3)+10; x3 = rt(10,3)+15
# Nota: Agrupe los datos y use la función ANOVA del modelo lm:
# Group = as.factor(rep(1:3, each=10)); datos = c(x1, x2, x3); anova(lm(datos ~ Group))

In [ ]:
import numpy as np
from scipy.stats import f
import pandas as pd
import pingouin as pg


In [ ]:
# datos aleatorios
np.random.seed(1)
x1 = np.random.randn(10, 3) + 10
x2 = np.random.randn(10, 3) + 10
x3 = np.random.randn(10, 3) + 10

x4 = np.random.randn(10, 3) + 10
x5 = np.random.randn(10, 3) + 10
x6 = np.random.randn(10, 3) + 15

In [ ]:
# los ponemos en un df al primer grupo
data1 = np.concatenate([x1, x2, x3]).flatten()
groups1 = np.repeat(np.arange(1, 4), 30)
df1 = pd.DataFrame(data = data1 , columns =['data'])
df1['groups'] = groups1
df1.head()

,data,groups
0,11.624345,1
1,9.388244,1
2,9.471828,1
3,8.927031,1
4,10.865408,1


In [ ]:
# Poner en un df el segundo grupo
data2 = np.concatenate([x4, x5, x6]).flatten()
groups2 = np.repeat(np.arange(1, 4), 30)
df2 = pd.DataFrame(data = data2 , columns =['data'])
df2['groups'] = groups2
df2.head()

,data,groups
0,11.624345,1
1,9.388244,1
2,9.471828,1
3,8.927031,1
4,10.865408,1


In [ ]:
#calculamos el anova para obtener el valor F
aov1 = pg.anova(data=df1, dv='data', between='groups', detailed=True)
print(aov1)
f_df1 = aov1.F.values[0]
print(f_df1)

   Source         SS  DF        MS         F     p-unc       np2
0  groups   0.762354   2  0.381177  0.445539  0.641932  0.010138
1  Within  74.432054  87  0.855541       NaN       NaN       NaN
0.4455393118475391


In [ ]:
#calculamos el anova para obtener el valor F

aov2 = pg.anova(data=df2, dv='data', between='groups', detailed=True)
print(aov2)
f_df2 = aov2.F.values[0]
print(f_df2)

   Source          SS  DF          MS           F         p-unc      np2
0  groups  531.837870   2  265.918935  310.819681  2.374587e-40  0.87723
1  Within   74.432054  87    0.855541         NaN           NaN      NaN
310.8196811556081


In [ ]:
# hacemos las permutaciones para grupo 1
num_permutations = 10000
count_extreme_1 = 0

for _ in range(num_permutations):
    #creo el df
    df_test_1 = df1
    np.random.shuffle(groups1) # reburujamos los grupos
    df_test_1['groups'] =  groups1
    # calculo el anova para obtener el F
    aov_test = pg.anova(data=df_test_1, dv='data', between='groups', detailed=True)
    try:
      f1_test = aov_test.F.values[0]
          # el valor F es mucho mas mayor que el observado de al principio
      if f1_test >= f_df1:
          count_extreme_1 += 1
    except:
      pass

# pvalor
p_value_1 = count_extreme_1 / num_permutations

print("Observed F statistic:", f_df1)
print("Number of permutations:", num_permutations)
print("Number of extreme permutations:", count_extreme_1)
print("P-value:", p_value_1)


Observed F statistic: 0.4455393118475391
Number of permutations: 10000
Number of extreme permutations: 6378
P-value: 0.6378


In [ ]:
# hacemos las permutaciones para grupo 2

num_permutations = 10000
count_extreme_2 = 0

for _ in range(num_permutations):
    #creo el df
    df_test_2= df2
    np.random.shuffle(groups2)# reburujamos las etiquetas de los grupos
    df_test_2['groups'] =  groups2
    # calculo el anova para obtener el F
    aov_test_2 = pg.anova(data=df_test_2, dv='data', between='groups', detailed=True)
    try:
      f1_test_2 = aov_test_2.F.values[0]
          # el valor F es mucho mas mayor que el observado de al principio
      if f1_test_2 >= f_df2:
          count_extreme_2 += 1
    except:
      pass

# Calculate the p-value
p_value_2 = count_extreme_2 / num_permutations

print("Observed F statistic:", f_df2)
print("Number of permutations:", num_permutations)
print("Number of extreme permutations:", count_extreme_2)
print("P-value:", p_value_2)

Observed F statistic: 310.8196811556081
Number of permutations: 10000
Number of extreme permutations: 0
P-value: 0.0
